# 0. 기존 DB자료 "Tab" → "," 변환

In [9]:
# ==============================================================================
#                  ★★ 최종 GPU 설정 및 라이브러리 임포트 셀 ★★
#               (노트북을 열면 무조건 이 셀부터 실행하세요)
# ==============================================================================

import tensorflow as tf
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path

def initialize_gpu():
    """
    TensorFlow GPU를 안전하게 초기화하고, 이미 초기화된 경우에도 정보를 제공합니다.
    """
    print("[INFO] TensorFlow GPU 확인을 시작합니다...")
    
    gpus = tf.config.list_physical_devices('GPU')
    
    if not gpus:
        print("[WARN] GPU를 찾을 수 없습니다. CPU로 실행됩니다.")
        return

    print(f"[INFO] 발견된 물리적 GPU: {len(gpus)}개")

    try:
        # 메모리 동적 할당을 '시도'합니다.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("[SUCCESS] GPU 메모리 동적 할당 설정에 성공했습니다!")
    
    except RuntimeError as e:
        # 만약 오류가 발생하면 (대부분 이미 초기화된 경우)
        print(f"[NOTE] GPU 메모리 설정 중 참고사항: {e}")
        print("       (VS Code의 자동 실행 등으로 이미 초기화되었을 수 있으나, GPU 사용에는 문제가 없을 가능성이 높습니다.)")

    # 최종적으로 TensorFlow가 실제로 사용할 수 있는 논리적 GPU 개수를 확인합니다.
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(f"[FINAL CHECK] TensorFlow가 인식하고 사용할 수 있는 GPU: {len(logical_gpus)}개")
    
    if len(logical_gpus) > 0:
        print("       >>> GPU가 성공적으로 인식되었습니다. 이제부터 GPU 연산이 가능합니다. <<<")
    else:
        print("       >>> [ERROR] GPU가 발견되었으나, TensorFlow가 사용할 수 없는 상태입니다. 드라이버나 설치를 확인하세요. <<<")

# 함수를 호출하여 GPU 설정을 실행합니다.
initialize_gpu()
# ==============================================================================

2025-08-20 16:07:13.910653: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-20 16:07:13.933990: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-20 16:07:25.545142: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


[INFO] TensorFlow GPU 확인을 시작합니다...
[INFO] 발견된 물리적 GPU: 1개
[SUCCESS] GPU 메모리 동적 할당 설정에 성공했습니다!
[FINAL CHECK] TensorFlow가 인식하고 사용할 수 있는 GPU: 1개
       >>> GPU가 성공적으로 인식되었습니다. 이제부터 GPU 연산이 가능합니다. <<<


I0000 00:00:1755673650.389777  377818 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21055 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
#%pip install numpy pandas matplotlib seaborn scikit-learn mglearn pyarrow tensorflow keras tqdm

# 1. 설정, 유틸 함수들

In [10]:
# ===== 셀 1: 설정 + 유틸 =====
import os, json, math
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import pandas as pd

# ▼ CSV 파일이 있는 폴더와 파일명(원하면 수정)
ROOT = "."
FILES = [("ygs.csv", "YGS"), ("kum.csv", "KUM"), ("hws.csv", "HWS"), ("icn.csv", "ICN")]

# ▼ 출력 폴더
OUTDIR = Path("preprocessed")
OUTDIR.mkdir(parents=True, exist_ok=True)

# --- CSV 읽기(자동 구분자 감지 + 컬럼명 정규화: 소문자/특수문자→_) ---
def sniff_read_csv(path: str) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        print(f"[INFO] 파일 없음: {path}")
        return None
    try:
        df = pd.read_csv(path, sep=None, engine="python")
        df.columns = (
            df.columns
            .str.strip()
            .str.lower()
            .str.replace(r"[^0-9a-zA-Z_]+", "_", regex=True)
        )
        return df
    except Exception as e:
        print(f"[ERROR] 읽기 실패 {path}: {e}")
        return None

# --- 데이터프레임 요약 출력 ---
def print_df_summary(name: str, df: pd.DataFrame, head_rows: int = 3) -> None:
    print(f"\n=== {name} | shape={df.shape} ===")
    print("Columns:", list(df.columns))
    print("Sample rows:")
    print(df.head(head_rows).to_string(index=False))

# --- 필수 컬럼 표준화(정확한 이름으로 강제) ---
#   주신 실제 컬럼 목록에 맞춰 '그대로' 매핑합니다.
#   - epc_code        ← 'epc_code'
#   - event_time      ← 'event_time'
#   - event_type      ← 'event_type'
#   - location_id     ← 'location_id'  (이름 유지; category로 사용)
REQUIRED = ["epc_code", "event_time", "event_type", "location_id"]

def standardize_required(df: pd.DataFrame) -> pd.DataFrame:
    # 이미 소문자/언더스코어 처리된 상태라는 가정하에, 필수 컬럼 확인
    missing = [c for c in REQUIRED if c not in df.columns]
    if missing:
        raise KeyError(f"필수 컬럼이 없습니다: {missing}\n"
                    f"현재 컬럼: {list(df.columns)}")
    # 불필요한 공백/형 변환 등은 아래 단계에서 처리
    return df

# --- 시간 파싱 + 시계열 피처(Δt, 시간대 sin/cos) ---
def ensure_dt(series: pd.Series) -> pd.Series:
    return pd.to_datetime(series, errors="coerce", utc=False)

def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    if not {"epc_code", "event_time"}.issubset(df.columns):
        raise KeyError("시간 피처 생성을 위해 'epc_code'와 'event_time'이 필요합니다.")
    out = df.copy()
    out["event_time"] = ensure_dt(out["event_time"])
    # EPC별 시간 정렬
    out.sort_values(["epc_code", "event_time"], inplace=True, kind="stable")
    # Δt(초) — 시퀀스 시작은 0
    out["delta_t_sec"] = out.groupby("epc_code")["event_time"].diff().dt.total_seconds().fillna(0.0)
    # 시간대(0~23) → sin/cos
    hours = out["event_time"].dt.hour.astype(float)
    out["hour_sin"] = (2 * math.pi * hours / 24.0).apply(math.sin)
    out["hour_cos"] = (2 * math.pi * hours / 24.0).apply(math.cos)
    return out

# --- 간단 어휘집 생성기 ---
def build_vocab(series: pd.Series, name: str, min_freq: int = 1) -> Dict:
    """
    - 예약 토큰: <PAD>=0, <UNK>=1
    - 정렬: 빈도 내림차순 → (동률 시) 토큰 문자열 오름차순
    - location_id는 숫자여도 '코드'이므로 문자열로 변환 후 카테고리로 취급합니다.
    """
    s = series.astype(str)
    counts = s.value_counts(dropna=False)
    counts = counts[counts >= min_freq]
    items = sorted(counts.items(), key=lambda kv: (-int(kv[1]), str(kv[0])))
    id2token = ["<PAD>", "<UNK>"] + [str(k) for k, _ in items]
    return {
        "name": name,
        "reserved": ["<PAD>", "<UNK>"],
        "id2token": id2token,
        "counts": {str(k): int(v) for k, v in counts.items()},
    }

# --- 어휘집을 이용해 ID 인코딩 ---
def encode_with_vocab(series: pd.Series, vocab: Dict, colname: str) -> pd.Series:
    token2id = {tok: i for i, tok in enumerate(vocab["id2token"])}
    unk_id = vocab["reserved"].index("<UNK>") if "<UNK>" in vocab["reserved"] else 1
    return series.astype(str).map(lambda x: token2id.get(x, unk_id)).astype("int32")

# --- 저장 도우미(Parquet 우선, 실패 시 CSV 폴백) ---
def save_table(df: pd.DataFrame, path_parquet: Path, path_csv: Optional[Path] = None):
    try:
        df.to_parquet(path_parquet, index=False)
        print(f"[OK] 저장(parquet): {path_parquet} (shape={df.shape})")
    except Exception as e:
        if path_csv is None:
            path_csv = path_parquet.with_suffix(".csv")
        df.to_csv(path_csv, index=False)
        print(f"[WARN] parquet 저장 실패({e}) → CSV로 저장: {path_csv} (shape={df.shape})")


# 2. CSV 읽기 & 요약

In [11]:
# ===== 셀 2: CSV 읽기 & 요약 =====
dfs = []
for fname, factory in FILES:
    path = os.path.join(ROOT, fname)
    df = sniff_read_csv(path)
    if df is None:
        continue
    print_df_summary(f"{fname} ({factory})", df)
    # factory가 데이터에 별도 컬럼으로 있을 수도 있지만, 파일별 라벨을 임시로 붙여둡니다.
    df["__factory_label__"] = factory
    dfs.append(df)

if not dfs:
    raise SystemExit("[STOP] CSV를 하나도 찾지 못했습니다. 파일 경로/이름을 확인하세요.")



=== ygs.csv (YGS) | shape=(132200, 18) ===
Columns: ['_scan_location', 'location_id', 'hub_type', 'business_step', 'event_type', 'operator_id', 'device_id', 'epc_code', 'epc_header', 'epc_company', 'epc_product', 'epc_lot', 'epc_manufacture', 'epc_serial', 'product_name', 'event_time', 'manufacture_date', 'expiry_date']
Sample rows:
_scan_location  location_id    hub_type business_step  event_type  operator_id  device_id                                      epc_code  epc_header  epc_company  epc_product  epc_lot  epc_manufacture  epc_serial product_name          event_time    manufacture_date  expiry_date
          양산공장            3 YGS_Factory       Factory Aggregation            3          3 001.8805843.2932031.100001.20250701.000000001           1      8805843      2932031   100001         20250701           1    Product 1 2025-07-01 10:23:39 2025-07-01 10:23:39     20251231
          양산공장            3 YGS_Factory       Factory Aggregation            3          3 001.8809437.120319

# 3. 병합 → 필수 컬럼 표준화 → 저장

In [14]:
# ===== 셀 3: 병합/표준화/저장 =====
merged = pd.concat(dfs, ignore_index=True)
print(f"\n[INFO] 병합 shape: {merged.shape}")

# 필수 컬럼 확인/고정(정확한 이름 사용)
std = standardize_required(merged)

# factory 최종화: 원본에 'factory'가 없다면, 임시 라벨 사용
if "factory" not in std.columns:
    std["factory"] = std.pop("__factory_label__")
else:
    std.drop(columns=["__factory_label__"], errors="ignore", inplace=True)

# [수정 제안] Parquet 저장을 위해 데이터 타입을 명시적으로 변환
# 문제가 될 만한 object 타입 컬럼을 문자열(str)로 바꿔줍니다.
for col in std.select_dtypes(include=['object']).columns:
    print(f"[DEBUG] Converting column '{col}' to string type for Parquet compatibility.")
    std[col] = std[col].astype(str)

# 저장
save_table(std, OUTDIR / "combined.parquet")



[INFO] 병합 shape: (920000, 19)
[DEBUG] Converting column '_scan_location' to string type for Parquet compatibility.
[DEBUG] Converting column 'hub_type' to string type for Parquet compatibility.
[DEBUG] Converting column 'business_step' to string type for Parquet compatibility.
[DEBUG] Converting column 'event_type' to string type for Parquet compatibility.
[DEBUG] Converting column 'epc_code' to string type for Parquet compatibility.
[DEBUG] Converting column 'product_name' to string type for Parquet compatibility.
[DEBUG] Converting column 'event_time' to string type for Parquet compatibility.
[DEBUG] Converting column 'manufacture_date' to string type for Parquet compatibility.
[DEBUG] Converting column 'factory' to string type for Parquet compatibility.
[OK] 저장(parquet): preprocessed/combined.parquet (shape=(920000, 19))


# 4. 시간 피처(Δt, hour sin/cos) 생성 → 저장

In [15]:
# ===== 셀 4: 시간 피처 생성/저장 =====
std_time = add_time_features(std)

# --- [수정] Parquet 저장을 위한 임시 데이터프레임 생성 ---
# 원본 std_time의 타입을 바꾸지 않고, 저장을 위한 복사본을 만듭니다.
df_to_save = std_time.copy()

# datetime 타입은 종종 Parquet 변환 시 문제를 일으키므로,
# 가장 안전한 ISO 8601 표준 문자열 포맷으로 변환하여 저장합니다.
df_to_save["event_time"] = df_to_save["event_time"].astype(str)
# --------------------------------------------------------

save_table(std_time, OUTDIR / "combined_with_time.parquet")


[OK] 저장(parquet): preprocessed/combined_with_time.parquet (shape=(920000, 22))


# 5. 어휘집 생성(JSON) + ID 인코딩 컬럼 추가 → 저장

In [16]:
# ===== 셀 5: vocab 생성 + ID 인코딩 =====

# 1) event_type 어휘집
vocab_event = build_vocab(std_time["event_type"], name="event_type", min_freq=1)
with open(OUTDIR / "event_type.vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_event, f, ensure_ascii=False, indent=2)
print(f"[OK] 어휘집 저장: {OUTDIR / 'event_type.vocab.json'} (size={len(vocab_event['id2token'])})")

# 2) location_id 어휘집 (숫자지만 '코드'이므로 카테고리 취급)
vocab_loc = build_vocab(std_time["location_id"], name="location_id", min_freq=1)
with open(OUTDIR / "location_id.vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_loc, f, ensure_ascii=False, indent=2)
print(f"[OK] 어휘집 저장: {OUTDIR / 'location_id.vocab.json'} (size={len(vocab_loc['id2token'])})")

# 3) ID 인코딩 컬럼 추가
encoded = std_time.copy()
encoded["event_type_id"]  = encode_with_vocab(encoded["event_type"],  vocab_event, "event_type")
encoded["location_id_id"] = encode_with_vocab(encoded["location_id"], vocab_loc,   "location_id")

# 저장
save_table(encoded, OUTDIR / "combined_encoded.parquet")


[OK] 어휘집 저장: preprocessed/event_type.vocab.json (size=12)
[OK] 어휘집 저장: preprocessed/location_id.vocab.json (size=60)
[OK] 저장(parquet): preprocessed/combined_encoded.parquet (shape=(920000, 24))


# 6. 모델 학습을 위한 준비 - 데이터 분할 및 스케일링

In [17]:
# ==============================================================================
#      (수정된 최종 셀 6) 피처 엔지니어링 + 데이터 분할 + 스케일링
# ==============================================================================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# --- 1. 셀 5에서 최종적으로 생성된 데이터를 불러옵니다. ---
print("[INFO] 인코딩된 데이터를 불러옵니다.")
try:
    df_base = pd.read_parquet(OUTDIR / "combined_encoded.parquet")
except Exception as e:
    df_base = pd.read_csv(OUTDIR / "combined_encoded.csv")
    print(f"[WARN] Parquet 읽기 실패({e}), CSV 파일 사용.")

# --- 2. [신규] EPC 그룹 단위의 통계적 피처를 생성합니다. (Isolation Forest용) ---
print("\n[INFO] EPC 그룹 단위의 통계적 피처를 생성합니다...")
epc_group_stats = df_base.groupby('epc_code').agg(
    seq_length=('event_time', 'count'),
    n_unique_locations=('location_id', 'nunique'),
    delta_t_mean=('delta_t_sec', 'mean'),
    delta_t_std=('delta_t_sec', 'std'),
    delta_t_max=('delta_t_sec', 'max')
).reset_index()

# std가 없는 경우(시퀀스 길이가 1) NaN이 되므로 0으로 채웁니다.
epc_group_stats.fillna(0, inplace=True)

# 원본 데이터에 통계 피처를 병합하여 '최종 엔지니어링된 데이터'를 만듭니다.
df_engineered = pd.merge(df_base, epc_group_stats, on='epc_code', how='left')

print("[OK] 그룹 통계 피처 병합 완료.")
print_df_summary("Engineered DataFrame", df_engineered.head()) # .head()로 요약본만 출력


# --- 3. EPC 코드를 기준으로 데이터를 학습(Train)과 테스트(Test) 세트로 분할합니다. ---
# (df_engineered를 분할해야 모든 피처가 train/test에 포함됩니다.)
epc_codes = df_engineered["epc_code"].unique()
train_epcs, test_epcs = train_test_split(epc_codes, test_size=0.2, random_state=42)

train_df = df_engineered[df_engineered["epc_code"].isin(train_epcs)].copy()
test_df = df_engineered[df_engineered["epc_code"].isin(test_epcs)].copy()

print(f"\n[INFO] 데이터 분할 완료: Train={train_df.shape}, Test={test_df.shape}")


# --- 4. 스케일링을 수행합니다. ---
# 스케일링 할 컬럼 목록: 이제 그룹 통계 피처들도 포함시킵니다.
# (id 컬럼이나 sin/cos 변환된 컬럼은 보통 제외합니다.)
cols_to_scale = [
    'delta_t_sec', 
    'seq_length', 
    'n_unique_locations', 
    'delta_t_mean', 
    'delta_t_std', 
    'delta_t_max'
]
print(f"[INFO] 스케일링 대상 컬럼: {cols_to_scale}")

# StandardScaler를 생성하고 '학습 데이터'에만 fit합니다.
scaler = StandardScaler()
print(f"[INFO] 학습 데이터에 스케일러를 학습(fit)합니다.")
scaler.fit(train_df[cols_to_scale])

# 학습된 스케일러로 Train, Test 데이터 모두를 transform합니다.
print("[INFO] 학습된 스케일러로 Train/Test 데이터를 변환(transform)합니다.")
train_df[cols_to_scale] = scaler.transform(train_df[cols_to_scale])
test_df[cols_to_scale] = scaler.transform(test_df[cols_to_scale])

# 학습된 스케일러를 파일로 저장합니다. (모델 재사용을 위해 필수!)
scaler_path = OUTDIR / "if_scaler.joblib" # Isolation Forest용 스케일러이므로 이름 변경
joblib.dump(scaler, scaler_path)
print(f"[OK] 스케일러 저장 완료: {scaler_path}")


# --- 5. 최종 데이터셋을 각각 저장합니다. ---
# 이제 이 데이터는 예측 모델과 Isolation Forest 모두를 위한 준비가 되었습니다.
save_table(train_df, OUTDIR / "train_final_engineered.parquet")
save_table(test_df, OUTDIR / "test_final_engineered.parquet")

print("\n--- 스케일링 결과 확인 (Train Set) ---")
print(train_df[cols_to_scale].describe())

[INFO] 인코딩된 데이터를 불러옵니다.

[INFO] EPC 그룹 단위의 통계적 피처를 생성합니다...
[OK] 그룹 통계 피처 병합 완료.

=== Engineered DataFrame | shape=(5, 29) ===
Columns: ['_scan_location', 'location_id', 'hub_type', 'business_step', 'event_type', 'operator_id', 'device_id', 'epc_code', 'epc_header', 'epc_company', 'epc_product', 'epc_lot', 'epc_manufacture', 'epc_serial', 'product_name', 'event_time', 'manufacture_date', 'expiry_date', 'factory', 'delta_t_sec', 'hour_sin', 'hour_cos', 'event_type_id', 'location_id_id', 'seq_length', 'n_unique_locations', 'delta_t_mean', 'delta_t_std', 'delta_t_max']
Sample rows:
_scan_location  location_id         hub_type business_step   event_type  operator_id  device_id                                      epc_code  epc_header  epc_company  epc_product  epc_lot  epc_manufacture  epc_serial product_name          event_time    manufacture_date  expiry_date factory  delta_t_sec  hour_sin  hour_cos  event_type_id  location_id_id  seq_length  n_unique_locations  delta_t_mean  delta_t_std

# 7. Isolation Forest

In [18]:
# ==============================================================================
#      ★★ (신규 셀 7) Isolation Forest 모델 훈련 및 저장 ★★
#     (이 셀은 전처리 마지막 셀(셀 6)이 완료된 후에 실행합니다)
# ==============================================================================
import pandas as pd
import joblib
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report # 훈련 데이터에 대한 간이 테스트용

# --- 1. 전처리된 최종 '학습' 데이터 로드 ---
# Isolation Forest는 '정상' 데이터로만 훈련하는 것이 일반적이므로, train 셋만 필요합니다.
print("[INFO] 전처리된 학습 데이터를 불러옵니다...")
try:
    train_df = pd.read_parquet(OUTDIR / "train_final_engineered.parquet")
    print(f"[OK] {OUTDIR / 'train_final_engineered.parquet'} 로드 완료 (shape={train_df.shape})")
except Exception as e:
    print(f"[ERROR] 파일 로드 실패: {e}")
    # 파일이 없을 경우 스크립트 중단
    raise SystemExit("[STOP] 훈련 데이터 파일을 찾을 수 없습니다. 셀 6을 먼저 실행하세요.")

    
# --- 2. 모델 훈련에 사용할 특징(Features) 정의 ---
# 전처리 단계에서 스케일링했던 모든 특징을 그대로 사용합니다.
IF_FEATURES = [
    # 원본 특징 (스케일링 되지 않은 ID, sin/cos)
    'event_type_id', 
    'location_id_id', 
    'hour_sin', 
    'hour_cos',
    # 스케일링 된 수치형 특징
    'delta_t_sec',
    'seq_length',           
    'n_unique_locations',   
    'delta_t_mean',         
    'delta_t_std',          
    'delta_t_max'
]
# 누락된 컬럼이 있는지 최종 확인
missing_cols = [col for col in IF_FEATURES if col not in train_df.columns]
if missing_cols:
    raise ValueError(f"필요한 피처 중 일부가 데이터에 없습니다: {missing_cols}")
    
train_X_if = train_df[IF_FEATURES]


# --- 3. Isolation Forest 모델 생성 및 훈련 ---
print("\n[INFO] Isolation Forest 모델을 훈련합니다...")
# 하이퍼파라미터 설정:
#   - n_estimators: 사용할 나무(모델)의 개수. 많을수록 안정적이지만 훈련 시간이 길어짐.
#   - contamination: 훈련 데이터에 포함될 것으로 예상되는 이상치의 비율. 
#     'auto'는 보수적인 기준이며, 약간의 이상치를 허용하려면 0.01 (1%) 등으로 설정 가능.
#   - random_state: 재현성을 위해 난수 시드 고정.
#   - n_jobs=-1: 컴퓨터의 모든 CPU 코어를 사용하여 훈련 속도 향상.
iso_forest = IsolationForest(
    n_estimators=100,
    contamination='auto', 
    random_state=42, 
    n_jobs=-1,
    verbose=1 # 훈련 과정을 보여줌
)

# 모델 훈련 (fit)
iso_forest.fit(train_X_if)
print("[OK] Isolation Forest 모델 훈련 완료.")


# --- 4. 훈련된 모델 저장 ---
# LSTM 모델과 마찬가지로, 훈련된 Isolation Forest 모델을 저장하여
# 나중에 평가 노트북에서 재사용합니다.
model_path = OUTDIR / "iso_forest_model.joblib"
joblib.dump(iso_forest, model_path)
print(f"\n[SUCCESS] 훈련된 Isolation Forest 모델을 성공적으로 저장했습니다: {model_path}")


# --- 5. (선택 사항) 훈련 데이터 자체에 대한 간이 성능 확인 ---
print("\n--- [참고] 훈련 데이터에 대한 간이 성능 테스트 ---")
# 훈련 데이터에 대한 예측. -1이 이상치, 1이 정상
predictions_on_train = iso_forest.predict(train_X_if)

# -1(이상), 1(정상)을 1, 0으로 변환
predictions_binary = np.where(predictions_on_train == -1, 1, 0)
print("훈련 데이터에서 '이상'으로 감지된 샘플 수:", np.sum(predictions_binary))
print("훈련 데이터에서 '정상'으로 감지된 샘플 수:", len(predictions_binary) - np.sum(predictions_binary))
print(f"(contamination='auto' 설정에 따라 약 {len(predictions_binary) * 0.05:.0f}개 내외의 이상치를 감지하는 것이 일반적입니다.)")

[INFO] 전처리된 학습 데이터를 불러옵니다...
[OK] preprocessed/train_final_engineered.parquet 로드 완료 (shape=(736188, 29))

[INFO] Isolation Forest 모델을 훈련합니다...


[Parallel(n_jobs=32)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   2 out of  32 | elapsed:    0.2s remaining:    2.3s
[Parallel(n_jobs=32)]: Done  32 out of  32 | elapsed:    0.2s finished


[OK] Isolation Forest 모델 훈련 완료.

[SUCCESS] 훈련된 Isolation Forest 모델을 성공적으로 저장했습니다: preprocessed/iso_forest_model.joblib

--- [참고] 훈련 데이터에 대한 간이 성능 테스트 ---


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.4s


훈련 데이터에서 '이상'으로 감지된 샘플 수: 369517
훈련 데이터에서 '정상'으로 감지된 샘플 수: 366671
(contamination='auto' 설정에 따라 약 36809개 내외의 이상치를 감지하는 것이 일반적입니다.)


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished


# 8. 앙상블 각각의 모델을 개별적으로 학습

# 성능 테스트